#00. Imports

In [ ]:
!pip install yahoo_fin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 8.9 MB/s 
     |████████████████████████████████| 83 kB 2.1 MB/s 
     |████████████████████████████████| 127 kB 40.3 MB/s 
     |████████████████████████████████| 112 kB 63.1 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=7a8315f0eed1adc0ef0b2d81ff8ace99157667b9f80c6706d9e089080b8ede55
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=11734e4a74c3e53fcd5ec4fb6c5479cc7856c6c9df0153bc051c63d59ef76ebe
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=ecbcf0845a359e6c098b2adcde5b794621813d9de0bc2ae04f170

In [216]:
from google.colab import drive
from keras.models import load_model
import tensorflow as tf
import pandas as pd
from tensorflow import keras as k
from yahoo_fin import stock_info as si
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [217]:
import pandas as pd

In [218]:
import numpy as np

In [219]:
from sklearn.preprocessing import StandardScaler

#A. Load Models

Contains models of LSTM of variants Boole and Albert

##1. Load Model v11[Albert]

In [220]:
def rmse(y_true, y_pred):
  return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

In [221]:
model_albert = load_model("/content/drive/MyDrive/Models/Model_V11.h5", custom_objects={'loss': rmse }, compile=False)

In [222]:
model_albert.compile(loss = rmse, optimizer= k.optimizers.Adam(learning_rate=0.01))
early_stopping = k.callbacks.EarlyStopping(monitor='loss', patience=10)
model_albert.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 24, 50)            11200     
                                                                 
 lstm_22 (LSTM)              (None, 24, 50)            20200     
                                                                 
 lstm_23 (LSTM)              (None, 50)                20200     
                                                                 
 dropout_6 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 24)                1224      
                                                                 
Total params: 52,824
Trainable params: 52,824
Non-trainable params: 0
_________________________________________________________________


##2. Load Model v2[Boole]

In [223]:
model_boole = load_model("/content/drive/MyDrive/Models/Model_V2", custom_objects={'loss': rmse }, compile=False)

In [224]:
model_boole.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 24, 240)           235200    
                                                                 
 lstm_19 (LSTM)              (None, 24, 120)           173280    
                                                                 
 lstm_20 (LSTM)              (None, 60)                43440     
                                                                 
 dropout_6 (Dropout)         (None, 60)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 61        
                                                                 
Total params: 451,981
Trainable params: 451,981
Non-trainable params: 0
_________________________________________________________________


#B. Prepare and load Data

Contains methods to gather and preprocess data

In [225]:
df = si.get_data('NG=F')['close']

In [226]:
df = df.round(3)
df = df.resample('1D').mean()
df.interpolate(method='linear', axis = 0, inplace= True, limit_direction='forward')

In [227]:
df.head(5)

2000-08-30    4.80500
2000-08-31    4.78000
2000-09-01    4.83500
2000-09-02    4.86625
2000-09-03    4.89750
Freq: D, Name: close, dtype: float64

##1. Data Prep v11[Albert]

In [228]:
df_m_albert = pd.DataFrame(df.resample('1M').mean())

In [229]:
type(df_m_albert)

pandas.core.frame.DataFrame

In [230]:
df_m_albert['std'] = df_m_albert['close'].rolling(10).std()

In [231]:
df_m_albert.head(15)

,close,std
2000-08-31,4.792500,NaN
2000-09-30,5.122239,NaN
2000-10-31,5.069688,NaN
2000-11-30,5.800333,NaN
2000-12-31,8.706976,NaN
2001-01-31,7.939685,NaN
2001-02-28,5.678107,NaN
2001-03-31,5.194559,NaN
2001-04-30,5.196172,NaN
2001-05-31,4.217532,1.425124


In [232]:
df_m_albert = df_m_albert[9:]

In [233]:
df_m_albert['std'].isna().sum()

0

In [234]:
df_m_albert['close_max'] = df_m_albert['close'] + df_m_albert['std']
df_m_albert['close_mim'] = df_m_albert['close'] - df_m_albert['std']

In [235]:
df_m_albert['gradient'] = np.gradient(df_m_albert['close'])
df_m_albert['d_gradient'] = np.gradient(df_m_albert['gradient'])

In [236]:
df_m_albert.head(5)

,close,std,close_max,close_mim,gradient,d_gradient
2001-05-31,4.217532,1.425124,5.642656,2.792409,-0.421032,-0.107306
2001-06-30,3.796500,1.531968,5.328468,2.264532,-0.528339,-0.004181
2001-07-31,3.160855,1.723754,4.884609,1.437101,-0.429395,0.045318
2001-08-31,2.937710,1.902204,4.839914,1.035506,-0.437702,0.135520
2001-09-30,2.285450,2.105705,4.391155,0.179745,-0.158355,0.348322


In [237]:
scaler_albert = StandardScaler()
scaler_albert_obj = scaler_albert.fit(df_m_albert)
scaler_albert_mean = list(map(np.float32, scaler_albert.mean_))
scaler_albert_var = scaler_albert.scale_

In [238]:
df_albert = scaler_albert.transform(df_m_albert)

In [239]:
type(df_albert)

numpy.ndarray

##2. Data Prep v2[Boole]

In [240]:
df_d_boole = pd.DataFrame(df.resample('1D').mean())

In [241]:
df_d_boole.interpolate(method='linear', axis = 0, inplace= True, limit_direction='forward')

In [242]:
df_d_boole['ma180'] = df_d_boole['close'].rolling(180).mean()
df_d_boole['ma60'] = df_d_boole['close'].rolling(60).mean()
df_d_boole['ma30'] = df_d_boole['close'].rolling(30).mean()

In [243]:
df_d_boole = df_d_boole[270:]

In [244]:
df_m_boole = df_d_boole.resample('1M').mean()

In [245]:
scaler_boole = StandardScaler()
scaler_boole_obj = scaler_boole.fit(df_m_boole)

In [246]:
df_m_boole = scaler_boole.transform(df_m_boole)

In [247]:
df_boole = df_m_boole

In [248]:
type(df_m_boole)

numpy.ndarray

#C. Batch Generator

Contains methods to transform and batch segment data for models

In [249]:
def batch_gen(df, size_x = 24, size_y = 12):
  X = []
  Y = []
  for i in range(size_x, len(df) - size_y):
    X.append(df[i-size_x:i])
    Y.append([item[0] for item in df[i:i+size_y]])

  X = np.array(X)
  Y = np.array(Y)
  return  X, Y

In [250]:
df_boole_X , df_boole_Y = batch_gen(df_boole , 24, 24)
df_albert_X , df_albert_Y = batch_gen(df_albert , 24, 24)

In [256]:
print(type(df_boole_X) ,type(df_boole_Y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [257]:
print(type(df_albert_X),type(df_albert_Y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [251]:
df_boole_X.shape

(208, 24, 4)

In [252]:
df_albert_X.shape

(208, 24, 6)

#D. Boosting

In [253]:
ratio_albert = 0.50
ratio_boole = 0.50

In [254]:
# def predict(df_albert, df_boole, ratio_albert = 0.50, ratio_boole = 0.50):
#   if type(df_albert) != 